In [44]:
import os
import urllib.request
import json
import openai
import random
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy.orm import Session, DeclarativeBase, MappedAsDataclass, Mapped, mapped_column
from sqlalchemy.types import JSON
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_TOKEN")

In [45]:
import replicate

model2version = {
    "text2image" : "f178fa7a1ae43a9a9af01b833b9d2ecf97b1bcb0acfd2dc5dd04895e042863f1",
    "image2image" : "15a3689ee13b0d2616e98820eca31d4c3abcd36672df6afce5cb6feb1d66087d",
    "image2text" : "a4a8bafd6089e1716b06057c42b19378250d008b80fe87caa5cd36d40c1eda90",
}

text2image = replicate.models.get("stability-ai/stable-diffusion")
text2image_version = text2image.versions.list()[0]

image2image = replicate.models.get("stability-ai/stable-diffusion-img2img")
image2image_version = image2image.versions.list()[0]

image2text = replicate.models.get("pharmapsychotic/clip-interrogator")
image2text_version = image2text.versions.list()[0]

version2model = {}

for model, label in [(text2image, 'text2image'), (image2image,'image2image'),(image2text, 'image2text')]:
    version2model.update({ver.id: label for ver in model.versions.list()})


## Constants

In [56]:
IMAGE_SAVE_PATH='/var/www/output.designresearch.works/'
ALLOWED_FILE_SIZES= [128, 256, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024]
SEED_RANGE=[0,9999999]
DATABASE_LOCATION='../classroom_backend_database.sqlite'

# Set up database

In [57]:
print(f"sqlite:///{DATABASE_LOCATION}")
engine = sqlalchemy.create_engine(f"sqlite:///{DATABASE_LOCATION}", echo=True)

class Base(DeclarativeBase, MappedAsDataclass):
    type_annotation_map = {
        dict: JSON
    }

class Payload(Base):
    __tablename__ = "payload"
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True, init=False)
    json: Mapped[dict]
    
Base.metadata.create_all(engine)

sqlite:///../classroom_backend_database.sqlite
2023-05-13 13:09:17,695 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-13 13:09:17,697 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("payload")
2023-05-13 13:09:17,699 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-13 13:09:17,703 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("payload")
2023-05-13 13:09:17,704 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-13 13:09:17,709 INFO sqlalchemy.engine.Engine 
CREATE TABLE payload (
	id INTEGER NOT NULL, 
	json JSON NOT NULL, 
	PRIMARY KEY (id)
)


2023-05-13 13:09:17,711 INFO sqlalchemy.engine.Engine [no key 0.00194s] ()
2023-05-13 13:09:17,719 INFO sqlalchemy.engine.Engine COMMIT


### Endpoints: /heartbeat

#### Input:
None

#### Output:
{ \
    "status": string \
}

In [28]:
# GET /heartbeat
print(json.dumps({'status': 'alive'}))

{"status": "alive"}


In [29]:
# ResponseInfo GET /heartbeat
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: /text_to_text

#### Input:
{ \
    "prompt": string, \
    "max_tokens": int (default: 100), \
    "temperature": float (defaut: 0.0) \
} 

#### Output:
{ \
    "completion": string \
}

In [30]:
# POST /text_to_text
req = json.loads(REQUEST)
body = req['body']

completion_object = openai.Completion.create(
  model="text-davinci-003",
  prompt=body['prompt'],
  max_tokens=body['max_tokens'] if 'max_tokens' in body else 100,
  temperature=body['temperature'] if 'temperature' in body else 0
)

print(json.dumps({
    'completion': completion_object.choices[0].text
}))

NameError: name 'REQUEST' is not defined

In [31]:
# ResponseInfo POST /text_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: /text_to_image

#### Input:
{ \
    "prompt": string, \
    "cfg_scale": float, \
    "init_img_url": string (URL of the image, default: None), \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1), \
    "width": int (default: 512), \
    "height": int (default: 512), \
    "num_inference_steps": int (default: 1), \
    "seed": int (default random) \,
    "negative_prompt": str (default "")
} 

#### Output:
{ \
    "image_url": string, \
    "seed": int \
}

In [32]:
# POST /text_to_image
req = json.loads(REQUEST)
body = req['body']

if "width" not in body or body["width"] not in ALLOWED_FILE_SIZES:
    body["width"] = 512

if "height" not in body or body["height"] not in ALLOWED_FILE_SIZES:
    body["height"] = 512
    
if body["height"] * body["height"] > 1024*768:
    body["height"] = 512
    body["width"] = 512
    
seed = body["seed"] if "seed" in body else random.randint(*SEED_RANGE)

output = text2image_version.predict(
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    init_image=body['init_img_url'] if 'init_img_url' in body else None,
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1,
    height=body["height"],
    width=body["width"],
    num_inference_steps=body["num_inference_steps"] if "num_inference_steps" in body else 1,
    seed=seed,
    negative_prompt=body["negative_prompt"] if "negative_prompt" in body else ""
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output,
    'seed': seed
}))

NameError: name 'REQUEST' is not defined

In [33]:
# ResponseInfo POST /text_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: /image_to_image

#### Input:
{ \
    "image_url": string (URL of the image), \
    "prompt": string, \
    "cfg_scale": float, \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1), \
    "width": int (default: 512), \
    "height": int (default: 512), \
    "num_inference_steps": int (default: 1), \
    "seed": int (default: random) \
} 

#### Output:
{ \
    "image_url": string, \
    "seed": int \
}

In [34]:
# POST /image_to_image
req = json.loads(REQUEST)
body = req['body']

if "width" not in body or body["width"] not in ALLOWED_FILE_SIZES:
    body["width"] = 512

if "height" not in body or body["height"] not in ALLOWED_FILE_SIZES:
    body["height"] = 512
    
if body["height"] * body["height"] > 1024*768:
    body["height"] = 512
    body["width"] = 512
    
seed = body["seed"] if "seed" in body else random.randint(*SEED_RANGE)

output = image2image_version.predict(
    image=body['image_url'],
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1,
    height=body["height"],
    width=body["width"],
    num_inference_steps=body["num_inference_steps"] if "num_inference_steps" in body else 1,
    seed=seed
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output,
    'seed': seed
}))

NameError: name 'REQUEST' is not defined

In [35]:
# ResponseInfo POST /image_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: /image_to_text

#### Input:
{ \
    "image_url": string (URL of the image), \
} 

#### Output:
{ \
    "image_url": string \
}

In [36]:
# POST /image_to_text
req = json.loads(REQUEST)
body = req['body']

output = image2text_version.predict(
    image=body['image_url'],
    clip_model_name='ViT-H-14/laion2b_s32b_b79k',
    mode='fast'
)

print(json.dumps({
    'image_url': output
}))

NameError: name 'REQUEST' is not defined

In [37]:
# ResponseInfo POST /image_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: /predictions

#### Input:
None

#### Output:
{ \
  'completed_at': string[time], \
  'created_at': string[time], \
  'error': None, \
  'id': string, \
  'input': dict[string, Any] \
  'metrics': {'predict_time': int }, \
  'output': list[string[url]], \
  'started_at': string[time], \
  'source': string[web|api], \
  'status': string, \
  'urls': { \
    'get': string[url], \
    'cancel': string[url] \
          }, \
  'version': string, \
  'webhook_completed': None, \
  'model': string[text2image|image2image|image2text] \
  }

In [38]:
# GET /predictions
res = replicate.predictions._client._request("GET", f"/v1/predictions").json()
predictions = res["results"]
while res['next'] is not None:
    next_url = res["next"][len('https://api.replicate.com'):]
    res = replicate.predictions._client._request("GET", next_url).json()
    predictions += res["results"]

valid_predictions = []
for pred in predictions:
    if pred['version'] in version2model:
        pred['model'] = version2model[pred['version']]
        valid_predictions.append(pred)

print(json.dumps(valid_predictions))
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [39]:
# ResponseInfo GET /predictions
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: GET /store

#### Input:
JSON-formatted string

#### Output:
None

In [40]:
# GET /store
with Session(engine) as session:
    entries = [{**entry.json, **{'database_id': entry.id}} for entry in session.query(Payload).all()]

print(json.dumps(entries))

2023-05-13 12:22:01,128 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-13 12:22:01,134 INFO sqlalchemy.engine.Engine SELECT payload.id AS payload_id, payload.json AS payload_json 
FROM payload
2023-05-13 12:22:01,135 INFO sqlalchemy.engine.Engine [generated in 0.00182s] ()
2023-05-13 12:22:01,140 INFO sqlalchemy.engine.Engine ROLLBACK
[]


In [20]:
# ResponseInfo GET /store
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Endpoints: POST /store

#### Input:
JSON-formatted string

#### Output:
None

In [ ]:
# POST /store

req = json.loads(REQUEST)
body = req['body']

with Session(engine) as session:
    new_entry = Payload(json=body)
    session.add(new_entry)
    session.commit()

In [42]:
# ResponseInfo POST /store

print(json.dumps({
    "status" : 200
}))

{"status": 200}


### Endpoints: DELETE /store

#### Input:
{ \
  'database_id': int, \
}

#### Output:
None

In [43]:
# DELETE /store

req = json.loads(REQUEST)
body = req['body']

with Session(engine) as session:
    entry_to_delete = session.get(Payload, body['database_id'])
    session.delete(entry_to_delete)
    session.commit()

NameError: name 'REQUEST' is not defined

In [ ]:
# ResponseInfo DELETE /store

print(json.dumps({
    "status" : 200
}))